In [1]:
# bob.refresh {"cron": "5,30 2-12 * * *", "timeout": 40000}
# bob.static {"token": "98a517f5b880eeda7ed8568d7e8fb77834ac95284fede5c0459b5d2f8192", "path": "/home/ftp/ftp/dev/wsr-public/data_output/Confirmed.html", "filename": "../data_output/Confirmed.html"} 
# bob.static {"token": "7d0fe7cd6483514da087ccf755b574964d799167e7fa5a18962e5772b5c9", "path": "/home/ftp/ftp/dev/wsr-public/data_output/Confirmed Rate.html", "filename": "../data_output/Confirmed Rate.html"} 
# bob.static {"token": "efd045150687448d5dbdc2c0acb3e51b05e4ba28d2d29f4d533f29c14c2b", "path": "/home/ftp/ftp/dev/wsr-public/data_output/Deaths.html", "filename": "../data_output/Deaths.html"} 
# bob.static {"token": "d14c73baee025b68caae42cb577dbef7c665e74e0fa1b89458ea52a9aa3e", "path": "/home/ftp/ftp/dev/wsr-public/data_output/Fatality Rate.html", "filename": "../data_output/Fatality Rate.html"}

In [2]:
%run __init__.ipynb

Successfully connected to MongoDB


In [3]:
#--- Check if the data are already updated
filePath = f'{output_folder}US_CONFIRMED_MAP.csv'
if path.exists(filePath):
    df_check = pd.read_csv(filePath, sep=';',low_memory=False).drop('Unnamed: 0',axis=1)
    date_check = df_check[(df_check['Date'] == yesterday.strftime('%m/%d/%y'))]
    if len(date_check) > 0:
        print(f'Data already updated ! DateTime : {now}')
        %stop

In [4]:
#--Create Dataframe
cUS=pd.read_csv(output_folder + 'US_CONFIRMED.csv',sep=";").drop('Unnamed: 0',axis=1)
cols_to_keep=['UID','iso2', 'iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key','Province_State','Country_Region']
cUS= cUS.melt(id_vars=cols_to_keep,value_name='Confirmed',var_name='Date').groupby(['Date','Province_State'], as_index=False).agg({'Confirmed':'sum'})

dUS=pd.read_csv(output_folder + 'US_DEATHS.csv',sep=";").drop('Unnamed: 0',axis=1)
cols_to_keep=['UID','iso2', 'iso3','code3','FIPS','Admin2','Lat','Long_','Combined_Key','Province_State','Country_Region','Population']
dUS= dUS.melt(id_vars=cols_to_keep,value_name='Deaths',var_name='Date').groupby(['Date','Province_State'], as_index=False).agg({'Deaths':'sum','Population':'sum'})

dfUS=pd.merge(cUS,dUS, on=['Date','Province_State'], how='left').reset_index(drop=True)
dfUS.loc[(dfUS['Deaths'] != 0),'Fatality Rate'] = round((dfUS['Deaths']/dfUS['Confirmed']*100),2)
dfUS.loc[(dfUS['Confirmed'] != 0),'Confirmed Rate'] = round((dfUS['Confirmed']/dfUS['Population']*100),2)
dfUS.loc[(dfUS['Deaths'] != 0),'Deaths Rate'] = round((dfUS['Deaths']/dfUS['Population']*100),2)

dfUS['Date'] = pd.to_datetime(dfUS['Date'], format='%m/%d/%y').dt.strftime('%m/%d/%y')
dfUS_map = dfUS[(dfUS['Date'] == yesterday.strftime('%m/%d/%y')) & (dfUS['Population'] != 0) & (dfUS['Confirmed'] != 0)]
if len(dfUS_map) == 0:
    raise Exception(f'Data not updated ! DateTime : {now}')
else:
    dfUS_map.to_csv(output_folder + 'US_CONFIRMED_MAP.csv',sep=";")
dfUS_map.head(5)

,Date,Province_State,Confirmed,Deaths,Population,Fatality Rate,Confirmed Rate,Deaths Rate
5916,05/03/20,Alabama,7888,290,4903185,3.68,0.16,0.01
5917,05/03/20,Alaska,368,9,731545,2.45,0.05,0.00
5919,05/03/20,Arizona,8640,362,7278717,4.19,0.12,0.00
5920,05/03/20,Arkansas,3437,76,3017804,2.21,0.11,0.00
5921,05/03/20,California,54903,2216,39512223,4.04,0.14,0.01


In [5]:
#Highlighted KPIS
def domain_108(df):
    domain = df.copy()
    #Calcul US
    domain = domain.groupby('Date', as_index=False).agg({'Confirmed':'sum', 'Deaths':'sum' , 'Population':'sum'})
    domain.loc[(domain['Deaths'] != 0),'Fatality Rate'] = round((domain['Deaths']/domain['Confirmed']*100),2)
    domain.loc[(domain['Confirmed'] != 0),'Confirmed Rate'] = round((domain['Confirmed']/domain['Population']*100),2)
    domain.loc[(domain['Deaths'] != 0),'Deaths Rate'] = round((domain['Deaths']/domain['Population']*100),2)
    
    #Create DF for HKPIS
    cols_to_keep = ['Date']
    domain= domain.melt(id_vars=cols_to_keep,value_name='VALUE',var_name='KPI')
    
    #Calc variation
    kpis = domain['KPI'].drop_duplicates().values.tolist()
    final_domain = pd.DataFrame()
    for kpi in kpis:
        tmp_df = domain[domain['KPI'] == kpi].sort_values(by="Date", ascending=False).reset_index(drop=True)
        final_domain = final_domain.append(tmp_df)
    
    #Create df last + merge
    final_domain['D'] = (final_domain.index).astype(str)
    final_domain['D-1'] = (final_domain.index + 1).astype(str)
    cols_to_rename = {"D": "D-1",'VALUE':'VALUE_D-1'}
    df_last = final_domain[['D','KPI','VALUE']].rename(index=str, columns=cols_to_rename) 
    final_domain = final_domain.merge(df_last, on= ['D-1','KPI'], how= 'left').drop(['D','D-1'], axis=1)
    
    #Add fields 
    #SCENARIO
    final_domain['SCENARIO'] = pd.to_datetime(final_domain['Date'], format='%m/%d/%y').dt.strftime('%d/%m/%Y')
    
    #VARIATION
    final_domain['VAR'] = final_domain['VALUE'] - final_domain['VALUE_D-1']
    final_domain['VARP'] = final_domain['VAR'] / abs(final_domain['VALUE_D-1']) * 100
    
    #UNIT & PRECISION
    final_domain.loc[:,'UNIT_VALUE'] = ''
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate','UNIT_VALUE'] = ' %'
    final_domain.loc[:,'UNIT_VAR'] = ''
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate','UNIT_VAR'] = ' pts'
    final_domain.loc[:,'PRECISION_VALUE'] = ',.0f'
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate','PRECISION_VALUE'] = ',.2f'
    final_domain.loc[:,'PRECISION_VAR'] = '+,.0f'
    final_domain.loc[final_domain['KPI'].str[-4:] == 'Rate','PRECISION_VAR'] = '+,.3f'
    
    final_domain = final_domain.drop('Date',axis=1).dropna().reset_index(drop=True)
    return final_domain

domain108 = domain_108(dfUS)
bob.mongo.save_df(domain108,'108',db_app,True)
domain108

Dataframe 108 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.03382730484008789 secnds ---


,KPI,VALUE,VALUE_D-1,SCENARIO,VAR,VARP,UNIT_VALUE,UNIT_VAR,PRECISION_VALUE,PRECISION_VAR
0,Confirmed,1158040.00,1132539.00,03/05/2020,25501.00,2.251666,,,",.0f","+,.0f"
1,Confirmed,1132539.00,1103461.00,02/05/2020,29078.00,2.635163,,,",.0f","+,.0f"
2,Confirmed,1103461.00,1069424.00,01/05/2020,34037.00,3.182741,,,",.0f","+,.0f"
3,Confirmed,1069424.00,1039909.00,30/04/2020,29515.00,2.838229,,,",.0f","+,.0f"
4,Confirmed,1039909.00,1012582.00,29/04/2020,27327.00,2.698744,,,",.0f","+,.0f"
...,...,...,...,...,...,...,...,...,...,...
399,Deaths Rate,0.01,0.01,12/04/2020,0.00,0.000000,%,pts,",.2f","+,.3f"
400,Deaths Rate,0.01,0.01,11/04/2020,0.00,0.000000,%,pts,",.2f","+,.3f"
401,Deaths Rate,0.01,0.01,10/04/2020,0.00,0.000000,%,pts,",.2f","+,.3f"
402,Deaths Rate,0.01,0.01,09/04/2020,0.00,0.000000,%,pts,",.2f","+,.3f"


In [6]:
def analysis(variable):
    Min = dfUS[variable].min()
    Max = dfUS[variable].max()
    Avr = round(dfUS[variable].mean(),0)
    Sum = dfUS[variable].sum()
    to_print = f'{variable} : Min: {Min}, Max: {Max}, Average: {Avr}, Sum: {Sum}'
    return print(to_print)

In [7]:
def updateChartCss(filename, css_filename):
    map = None
    css = None
    with open(filename) as f:
        map = f.read()

    with open(css_filename) as f:
        css = f.read()
    if (map.find('id="cs_css"') != -1):
        print("to do")
    else:
        result = map.replace("<body>",f'<body><style id="cs_css">{css}</style>')
        with open(filename, "w") as f:
            f.write(result)
            f.close()

In [8]:
def create_map(df,geo,kpi,scale,colors):
    fig = px.choropleth_mapbox(df, geojson=geo, color=kpi,
                               locations="Province_State", featureidkey="properties.NAME",
                               range_color=scale,
                               color_continuous_scale=colors,
#                                animation_frame="Date",
                               mapbox_style="carto-positron", 
                               zoom=3, center={"lat": 37.0902, "lon": -95.7129},
                               labels={'Province_State':'State'}
                              )
    fig.update_layout(margin={"r":0,"l":0,},autosize=True)
    fig.write_html(output_folder + kpi + ".html")
    updateChartCss(f"{output_folder}{kpi}.html", f"{input_folder}map.css")
#     bob.makeStaticGettable(output_folder + kpi + ".html","dev")
#     return fig

In [9]:
#Confirmed
kpi='Confirmed'
scale = (100,250000)
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
analysis(kpi)
create_map(dfUS_map,states20m,kpi,scale,colors)

Confirmed : Min: 0, Max: 316415, Average: 4013.0, Sum: 23976381


In [10]:
#Confirmed Rate
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (0.1,1)
kpi='Confirmed Rate'
analysis(kpi)
create_map(dfUS_map,states20m,kpi,scale,colors)

Confirmed Rate : Min: 0.0, Max: inf, Average: inf, Sum: inf


In [11]:
#'Deaths'
colors = [
    [0.0, "#FFFFFF"],
    [0.01,"#B3D7D2"],
    [0.1, "#81B1BD"],
    [0.3, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (0,10000)
kpi='Deaths'
analysis(kpi)
create_map(dfUS_map,states20m,kpi,scale,colors)

Deaths : Min: 0, Max: 24708, Average: 208.0, Sum: 1241271


In [12]:
#Fatality Rate
colors = [
    [0.0, "#FFFFFF"],
    [0.25,"#B3D7D2"],
    [0.5, "#81B1BD"],
    [0.75, "#437F9D"],
    [1.0, "#00354E"]
]
scale = (0,10)
kpi='Fatality Rate'
analysis(kpi)
create_map(dfUS_map,states20m,kpi,scale,colors)

Fatality Rate : Min: 0.17, Max: 33.33, Average: 3.0, Sum: 8435.41


In [13]:
#html
confirmed = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/98a517f5b880eeda7ed8568d7e8fb77834ac95284fede5c0459b5d2f8192', 'KPI' : 'Confirmed'}]
confirmedRate = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/7d0fe7cd6483514da087ccf755b574964d799167e7fa5a18962e5772b5c9', 'KPI' : 'Confirmed Rate'}]
deaths = [{'URL' : 'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/efd045150687448d5dbdc2c0acb3e51b05e4ba28d2d29f4d533f29c14c2b', 'KPI' : 'Deaths'}]
fatalityRate = [{'URL' :'https://api.dev.cashstory.com/api/v1/notebook/bob_40cashstory_2Ecom/d14c73baee025b68caae42cb577dbef7c665e74e0fa1b89458ea52a9aa3e', 'KPI' : 'Fatality Rate'}]

list_kpis = confirmed + confirmedRate + deaths + fatalityRate
domain = pd.DataFrame(list_kpis)
domain.loc[:,'VALUE'] = """<iframe src='"""+ domain['URL'] +"""' frameborder='0' style='overflow:hidden;height:85vh;width:100%' height='100%' width='100%'></iframe> <style>.modebar{display: none!important;}</style>"""
if use_mongo:
    bob.mongo.save_df(domain,'109',db_app,True)
domain

Dataframe 109 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.018329858779907227 secnds ---


,URL,KPI,VALUE
0,https://api.dev.cashstory.com/api/v1/notebook/...,Confirmed,<iframe src='https://api.dev.cashstory.com/api...
1,https://api.dev.cashstory.com/api/v1/notebook/...,Confirmed Rate,<iframe src='https://api.dev.cashstory.com/api...
2,https://api.dev.cashstory.com/api/v1/notebook/...,Deaths,<iframe src='https://api.dev.cashstory.com/api...
3,https://api.dev.cashstory.com/api/v1/notebook/...,Fatality Rate,<iframe src='https://api.dev.cashstory.com/api...
